In [ ]:
!pip install beautifulsoup4 requests

In [50]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

In [ ]:
# Initialize language model
llm = pipeline("text-generation", model="distilgpt2")

# Function to scrape book data from Project Gutenberg
def scrape_books_data():
    url = "https://www.gutenberg.org/browse/scores/top"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find book titles and authors (simplified scraping)
    books_data = []
    for item in soup.select("ol li a"):
        title = item.get_text()
        link = "https://www.gutenberg.org" + item['href']
        books_data.append({"title": title, "link": link})
        # Collect limited items for demonstration
        if len(books_data) >= 50:
            break

    return books_data

In [52]:
# Function to find all books with a specific keyword
def find_books_with_keyword(keyword, books_data):
    matched_books = [book for book in books_data if keyword.lower() in book["title"].lower()]
    if matched_books:
        return "\n".join([f"Title: {book['title']}\nLink: {book['link']}" for book in matched_books])
    else:
        return "Sorry, I couldn't find any books with that keyword."

In [53]:
# Chatbot function
def chat_with_bot(user_input, books_data):
    if "book" in user_input.lower():
        # Extract keyword based on "about" or "with keyword"
        if "about" in user_input.lower():
            keyword = user_input.lower().split("about")[-1].strip()
        elif "with keyword" in user_input.lower():
            keyword = user_input.lower().split("with keyword")[-1].strip()
        else:
            keyword = user_input.strip()

        # Perform search with the extracted keyword
        return find_books_with_keyword(keyword, books_data)
    else:
        # Use model for general conversation
        response = llm(f"{user_input}", max_length=100, num_return_sequences=1, do_sample=True)
        return response[0]['generated_text'].strip()

In [54]:
# Scrape books data dynamically
books_data = scrape_books_data()
print("Books data loaded successfully.")

Books data loaded successfully.


In [ ]:
# Chat interaction
print("Welcome to the Book Chatbot! Ask me about books.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    response = chat_with_bot(user_input, books_data)
    print("Bot:", response)